In [1]:
# output from prepare_pdbbind
failed_pdbs = {'1h6e': 'HIS residue (35) has the wrong set of atoms', '1svh': 'HIS residue (78) has the wrong set of atoms', '3bho': 'HIS residue (19) has the wrong set of atoms', '3bc5': 'HIS residue (6) has the wrong set of atoms', '3fxv': 'HIS residue (36) has the wrong set of atoms', '3s74': 'HIS residue (0) has the wrong set of atoms', '3smq': 'HIS residue (39) has the wrong set of atoms', '2y4k': 'HIS residue (50) has the wrong set of atoms', '3sm0': 'HIS residue (9) has the wrong set of atoms', '4hai': 'HIS residue (43) has the wrong set of atoms', '4ehm': 'HIS residue (1) has the wrong set of atoms', '4ayt': 'HIS residue (36) has the wrong set of atoms', '3rv9': 'HIS residue (21) has the wrong set of atoms', '3uib': 'HIS residue (50) has the wrong set of atoms', '4ayw': 'HIS residue (31) has the wrong set of atoms', '4gv8': 'HIS residue (34) has the wrong set of atoms', '4bae': 'HIS residue (34) has the wrong set of atoms', '4mb9': 'HIS residue (38) has the wrong set of atoms', '4ezj': 'HIS residue (18) has the wrong set of atoms', '4j8t': 'HIS residue (31) has the wrong set of atoms', '4aw8': 'HIS residue (40) has the wrong set of atoms', '4kv9': 'HIS residue (45) has the wrong set of atoms', '4kva': 'HIS residue (47) has the wrong set of atoms', '4q2k': 'HIS residue (8) has the wrong set of atoms', '4ryl': 'HIS residue (35) has the wrong set of atoms', '4rrr': 'HIS residue (48) has the wrong set of atoms', '4rrq': 'HIS residue (23) has the wrong set of atoms', '4yxu': 'HIS residue (1) has the wrong set of atoms', '4zyv': 'HIS residue (3) has the wrong set of atoms', '5iui': 'HIS residue (4) has the wrong set of atoms', '5izu': 'HIS residue (2) has the wrong set of atoms', '4xuh': 'HIS residue (12) has the wrong set of atoms', '4yb6': 'HIS residue (43) has the wrong set of atoms', '5hx8': 'HIS residue (6) has the wrong set of atoms', '5gic': 'HIS residue (48) has the wrong set of atoms', '5j3s': 'HIS residue (4) has the wrong set of atoms', '5mrd': 'HIS residue (5) has the wrong set of atoms', '5q0x': 'HIS residue (36) has the wrong set of atoms', '5ngf': 'HIS residue (41) has the wrong set of atoms', '5ien': 'HIS residue (63) has the wrong set of atoms', '5one': 'HIS residue (49) has the wrong set of atoms', '5if6': 'HIS residue (36) has the wrong set of atoms', '5wqd': 'HIS residue (21) has the wrong set of atoms', '6gbx': 'HIS residue (13) has the wrong set of atoms', '5wkl': 'HIS residue (45) has the wrong set of atoms', '6do4': 'HIS residue (54) has the wrong set of atoms', '5ypw': 'HIS residue (4) has the wrong set of atoms', '6ck3': 'HIS residue (6) has the wrong set of atoms', '5wgq': 'HIS residue (14) has the wrong set of atoms', '6do5': 'HIS residue (55) has the wrong set of atoms', '6hjk': 'HIS residue (25) has the wrong set of atoms', '6ck6': 'HIS residue (7) has the wrong set of atoms', '6cmr': 'HIS residue (10) has the wrong set of atoms', '6cjy': 'HIS residue (7) has the wrong set of atoms', '6c5t': 'HIS residue (11) has the wrong set of atoms', '6qyo': 'HIS residue (19) has the wrong set of atoms', '6i4x': 'HIS residue (42) has the wrong set of atoms', '6edl': 'HIS residue (3) has the wrong set of atoms', '6r0x': 'HIS residue (61) has the wrong set of atoms', '6e0r': 'HIS residue (4) has the wrong set of atoms', '6ebw': 'HIS residue (4) has the wrong set of atoms'}
failed_pdb_list = list(failed_pdbs.keys())
print(len(failed_pdbs))
failed_pdbs['5ypw']

61


'HIS residue (4) has the wrong set of atoms'

In [2]:
# 3nzc fails because plip does not find an interaction partner - this is also true in the web service!
# therefore I exclude it
failed_pdb_list.append('3nzc')

In [3]:
print(failed_pdb_list)

['1h6e', '1svh', '3bho', '3bc5', '3fxv', '3s74', '3smq', '2y4k', '3sm0', '4hai', '4ehm', '4ayt', '3rv9', '3uib', '4ayw', '4gv8', '4bae', '4mb9', '4ezj', '4j8t', '4aw8', '4kv9', '4kva', '4q2k', '4ryl', '4rrr', '4rrq', '4yxu', '4zyv', '5iui', '5izu', '4xuh', '4yb6', '5hx8', '5gic', '5j3s', '5mrd', '5q0x', '5ngf', '5ien', '5one', '5if6', '5wqd', '6gbx', '5wkl', '6do4', '5ypw', '6ck3', '5wgq', '6do5', '6hjk', '6ck6', '6cmr', '6cjy', '6c5t', '6qyo', '6i4x', '6edl', '6r0x', '6e0r', '6ebw', '3nzc']


In [4]:
import train
import torch
from torch import nn
from data import PDBBindInteractionDataset
from torch.utils.data import DataLoader
from e3nn import o3
from model import InteractionPredictor

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
lp_pdbbind = pd.read_csv('LP_PDBBind.csv', index_col=0)# TODO: Fix the 61 failed pdbs and remove this exclusion filter
train_pdbs = [ x for x in list(lp_pdbbind[(lp_pdbbind['new_split'] == 'train') & lp_pdbbind.CL1 & ~lp_pdbbind.covalent].index) if x not in failed_pdb_list ]
test_pdbs = [ x for x in list(lp_pdbbind[(lp_pdbbind['new_split'] == 'test') & lp_pdbbind.CL1 & ~lp_pdbbind.covalent].index) if x not in failed_pdb_list ]

train_pdbs = ['2gst', '5fnu', '6gl8', '1a0q']
test_pdbs = train_pdbs

['hbond', 'pistacking', 'hydrophobic', 'pication', 'saltbridges', 'halogenbond']
for interaction_type in ['pistacking', 'hydrophobic', 'pication', 'saltbridges', 'halogenbond']:
    print("@@@@ ", interaction_type)
    inter_pred = InteractionPredictor(n_pattern_layers = 1, 
                                  radius = 15.0,
                                  irreps_input = o3.Irreps("1x0e"),
                                  irreps_message = o3.Irreps("1x0e"),
                                  pattern_spherical_harmonics_l = 1,
                                  irreps_node = o3.Irreps("1x0e"),
                                  node_embedding_size = 1, 
                                  node_emb_hidden_layers = [], 
                                  node_act = torch.relu,
                                  edge_embedding_size = 1,
                                  edge_emb_hidden_layers = [],
                                  edge_act = torch.relu,
                                  msg_weights_hidden_layers = [],
                                  msg_weights_act = torch.relu,
                                  node_update_hidden_layers = [], 
                                  node_update_act = torch.relu,
                                  basis_density_per_A = 1, 
                                  inter_spherical_harmonics_l = 1,
                                  inter_tp_weights_hidden_layers = [], 
                                  inter_tp_weights_act = torch.relu,
                                  irreps_out = o3.Irreps("1x0e"))
    print("Model weights:", sum(p.numel() for p in inter_pred.parameters() if p.requires_grad))

    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(inter_pred.parameters(), lr=1e-3, amsgrad=True)

    dataset_train = PDBBindInteractionDataset("pdbbind2020/", train_pdbs, interaction_type)
    dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=False, collate_fn=dataset_train.collate_fn, pin_memory=True, num_workers=1)
    dataset_test = PDBBindInteractionDataset("pdbbind2020/", test_pdbs, interaction_type)
    dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=dataset_test.collate_fn, pin_memory=True, num_workers=1)

    print("Training size:", len(dataset_train), "Test size:", len(dataset_test))

    train.train(1, 1, dataloader_train, dataloader_test, inter_pred, loss_fn, optimizer, save_weights=False)
    print("\n-------------------------------------------------------------\n")

@@@@  pistacking
Model weights: 47
Training size: 4 Test size: 4
Using cuda device.


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/iwe20/anaconda3/envs/torch/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/iwe20/anaconda3/envs/torch/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iwe20/anaconda3/envs/torch/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/iwe20/Projects/LearnableInteractionKernel/data.py", line 25, in __getitem__
    rec_graph = torch.load(path + "_rec.graph")
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iwe20/an

In [ ]:
%reload_ext autoreload
%autoreload 2
import data
pdb_list = ['5fnu']
data.load_pdbbind_interactions(pdb_list[0], 'pistacking')

pistack(proteinring=aromatic_ring(atoms=[<openbabel.pybel.Atom object at 0x7fdd7ca37290>, <openbabel.pybel.Atom object at 0x7fdd7ca37310>, <openbabel.pybel.Atom object at 0x7fdd7ca37390>, <openbabel.pybel.Atom object at 0x7fdd7ca37410>, <openbabel.pybel.Atom object at 0x7fdd7ca37490>, <openbabel.pybel.Atom object at 0x7fdd7ca37510>], orig_atoms=[<openbabel.pybel.Atom object at 0x7fdd7ca8bb90>, <openbabel.pybel.Atom object at 0x7fdd7ca8bf10>, <openbabel.pybel.Atom object at 0x7fdd7ca8bed0>, <openbabel.pybel.Atom object at 0x7fdd7ca8b190>, <openbabel.pybel.Atom object at 0x7fdd7ca90450>, <openbabel.pybel.Atom object at 0x7fdd7ca90350>], atoms_orig_idx=[1523, 1524, 1525, 1526, 1527, 1528], normal=array([ 0.56246768, -0.48762181, -0.6677238 ]), obj=<openbabel.openbabel.OBRing; proxy of <Swig Object of type 'OpenBabel::OBRing *' at 0x7fdd7ca8e790> >, center=[14.730833333333331, 60.31583333333334, 22.852999999999998], type='6-membered'), ligandring=aromatic_ring(atoms=[<openbabel.pybel.Atom 

tensor([[  64, 2244],
        [  64, 2245],
        [  64, 2246],
        [  64, 2247],
        [  64, 2248],
        [  64, 2249],
        [  65, 2244],
        [  65, 2245],
        [  65, 2246],
        [  65, 2247],
        [  65, 2248],
        [  65, 2249],
        [  66, 2244],
        [  66, 2245],
        [  66, 2246],
        [  66, 2247],
        [  66, 2248],
        [  66, 2249],
        [  67, 2244],
        [  67, 2245],
        [  67, 2246],
        [  67, 2247],
        [  67, 2248],
        [  67, 2249],
        [  68, 2244],
        [  68, 2245],
        [  68, 2246],
        [  68, 2247],
        [  68, 2248],
        [  68, 2249],
        [  69, 2244],
        [  69, 2245],
        [  69, 2246],
        [  69, 2247],
        [  69, 2248],
        [  69, 2249],
        [1522, 2217],
        [1522, 2218],
        [1522, 2219],
        [1522, 2220],
        [1522, 2221],
        [1522, 2222],
        [1522, 2223],
        [1522, 2224],
        [1522, 2226],
        [1